In [1]:
import pandas as pd

from pandas import DataFrame, Series
import requests
import re
import lxml
from bs4 import BeautifulSoup

import numpy as np

`참고자료:`
- askdjango > pandas
- 구글링

### 크롤링을 하기 위해 필요한 기본 사항

-----------------------------


실제로 데이터를 가져올 웹사이트 주소를 **chart_url** 와 같이 변수에 할당해둔다. 

서버에 Request를 보낼 때 __안정적인 데이터 확보__ 를 위해 User Agent 를 헤더에 담아 `URL주소와 headers` 를 같이 요청한다.

*headers 는 선택사항이지만 크롤링을 하기 위해 꼭 넣어서 서버에 요청하는 것을 권장

*그리고 headers(User Agent) 는 브라우저 중에서 호환성이 관대한 FireFox, Chrome 를 추천

In [2]:
chart_url = 'https://music.bugs.co.kr/chart'

headers = {
    'User_agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X x.y; rv:10.0) Gecko/20100101 Firefox/10.0'
}

### requests 모듈을 사용하여 Response 200 코드 받기

- `requests` 모듈을 사용하여 chrart_url을 가져오게 되면 HTTP Response 의 응답을 받게 된다. Response 200을 받으면 데이터 파싱이 가능한 상태

```python
> res = requests.get(chart_url, headers)
> res
# output <Response [200]>
```

In [3]:
res = requests.get(chart_url, headers=headers)
html = res.text

## pandas 를 사용하여 본격적으로 크롤링하기

**pandas** 의 자료구조인 `data_frame`과 `series` 로 크롤링 해보기

----------------------

- Data_frame 은 2차적 데이터 구조

- Series 는 1차적 데이터 구조: Series를 사용하는 이유는 2개의 데이터베이스를 합칠 경우에 index의 순서가 달라도 스마트하게 동일한 index 끼리의 처리를 할 수 있기 때문이다.


In [4]:
type(pd.read_html(html)) # 데이터베이스 테이블이 list 타입임을 확인

list

In [5]:
len(pd.read_html(html)) # 데이터베이스 테이블(리스트)가 3개임을 확인

3

### 내가 추출하고자 하는 차트의 테이블 값을 추출

In [6]:
chart_df = pd.read_html(html)[0] # 총 3개의 리스트 중에서 첫번째[0] 의 리스트 값을 추출
chart_df

,Unnamed: 0,순위,Unnamed: 2,Unnamed: 3,곡,아티스트,앨범,듣기,재생목록,내앨범,다운,영상,기타
0,NaN,1 0변동없음,NaN,곡정보,삐삐,아이유(IU),삐삐,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생,기타 기능
1,NaN,2 0변동없음,NaN,곡정보,Save (Feat. 팔로알토)(Prod. 코드 쿤스트),루피(Loopy),쇼미더머니 777 Episode 3,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생 불가,기타 기능
2,NaN,3 0변동없음,NaN,곡정보,Kiss and Make Up,Dua Lipa(두아 리파) Dua Lipa(두아 리파),Dua Lipa [Complete Edition],듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생 불가,기타 기능
3,NaN,4 0변동없음,NaN,곡정보,멋지게 인사하는 법 (Feat. 슬기 of Red Velvet),Zion.T,ZZZ,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생,기타 기능
4,NaN,5 0변동없음,NaN,곡정보,고백,양다일,고백,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생,기타 기능
5,NaN,6 0변동없음,NaN,곡정보,Good Day (Feat. 팔로알토) (Prod. 코드 쿤스트),pH-1 pH-1,쇼미더머니 777 Episode 1,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생 불가,기타 기능
6,NaN,7 0변동없음,NaN,곡정보,시간이 들겠지 (Feat. Colde),로꼬,시간이 들겠지,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생,기타 기능
7,NaN,8 1 계단 상승,NaN,곡정보,꿈처럼 내린,다비치,뷰티 인사이드 (JTBC 월화드라마) OST - Part.3,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생,기타 기능
8,NaN,9 1 계단 하락,NaN,곡정보,가을 타나 봐,바이브(Vibe),가을 타나 봐,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생 불가,기타 기능
9,NaN,10 0변동없음,NaN,곡정보,하루도 그대를 사랑하지 않은 적이 없었다,임창정,하루도 그대를 사랑하지 않은 적이 없었다,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생,기타 기능


### 크롤링한 데이터프레임의 행과 열을 파악하는 방법 ( .shape)

In [7]:
chart_df.shape # 100개의 row 와 13개의 columns 로 이루어져있는 것을 확인

(100, 13)

### columns 를 이용하여 데이터프레임의 열에 속한 필드를 확인 (.columns)

In [8]:
chart_df.columns

Index(['Unnamed: 0', '순위', 'Unnamed: 2', 'Unnamed: 3', '곡', '아티스트', '앨범', '듣기',
       '재생목록', '내앨범', '다운', '영상', '기타'],
      dtype='object')

### .set_index 를 이용하여 다시 정렬하기

내가 원하는 레코드의 값만을 지정해서 추출하고 싶을 때가 있는데, `columns=[]` 를 이용하여 추출하고자 하는 필드명을 지정하고, `set_index()`로 기준을 잡아 다시 출력할 수 있다.

-------------------------


In [9]:
pd.DataFrame(chart_df, columns=['순위','곡', '아티스트', '앨범']).set_index('곡')

,순위,아티스트,앨범
곡,,,
삐삐,1 0변동없음,아이유(IU),삐삐
Save (Feat. 팔로알토)(Prod. 코드 쿤스트),2 0변동없음,루피(Loopy),쇼미더머니 777 Episode 3
Kiss and Make Up,3 0변동없음,Dua Lipa(두아 리파) Dua Lipa(두아 리파),Dua Lipa [Complete Edition]
멋지게 인사하는 법 (Feat. 슬기 of Red Velvet),4 0변동없음,Zion.T,ZZZ
고백,5 0변동없음,양다일,고백
Good Day (Feat. 팔로알토) (Prod. 코드 쿤스트),6 0변동없음,pH-1 pH-1,쇼미더머니 777 Episode 1
시간이 들겠지 (Feat. Colde),7 0변동없음,로꼬,시간이 들겠지
꿈처럼 내린,8 1 계단 상승,다비치,뷰티 인사이드 (JTBC 월화드라마) OST - Part.3
가을 타나 봐,9 1 계단 하락,바이브(Vibe),가을 타나 봐


### 데이터프레임에 '좋아요' 필드를 추가하고 head, tail 을 사용해보기

- `.head()` 의 기본 추출은 상위 5개의 필드값만 나오며, 10개만 추출하고 싶다면 .head(10)과 같은 옵션을 설정

- `.tail()` 역시 기본 추출은 하위 5개의 필드값이며, 10개만 추출하고 싶다면 .tail(10) 옵션을 설정

데이터프레임에 필드를 추가하는 방식은 파이썬에서 dict 를 추가하는 방식과 비슷하다.

In [10]:
color_dict = {'blue': '파란색', 'red': '빨간색', 'yellow': '노란색'}

In [11]:
color_dict

{'blue': '파란색', 'red': '빨간색', 'yellow': '노란색'}

In [12]:
color_dict['black'] = '검정색' # dict 타입에 key, value 추가

In [13]:
color_dict

{'blue': '파란색', 'red': '빨간색', 'yellow': '노란색', 'black': '검정색'}

dict 타입에 key, value 를 추가한 것처럼 DataFrame 에 추가해보자.

In [14]:
chart_df['좋아요'] = 100 # 최초에 좋아요 레코드값을 100으로 통일하여 설정

In [15]:
# 정렬 기준에 따라서 레코드값을 1 ~ 101까지 설정
# 만약 데이터프레임과 설정하려는 레코드값이 다를 경우에는 Length 에러가 발생
## chart_df['좋아요'] = range(1, 30) <-- Length 에러

chart_df['좋아요'] = range(1, 101)

In [16]:
chart_df.head(10)

,Unnamed: 0,순위,Unnamed: 2,Unnamed: 3,곡,아티스트,앨범,듣기,재생목록,내앨범,다운,영상,기타,좋아요
0,NaN,1 0변동없음,NaN,곡정보,삐삐,아이유(IU),삐삐,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생,기타 기능,1
1,NaN,2 0변동없음,NaN,곡정보,Save (Feat. 팔로알토)(Prod. 코드 쿤스트),루피(Loopy),쇼미더머니 777 Episode 3,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생 불가,기타 기능,2
2,NaN,3 0변동없음,NaN,곡정보,Kiss and Make Up,Dua Lipa(두아 리파) Dua Lipa(두아 리파),Dua Lipa [Complete Edition],듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생 불가,기타 기능,3
3,NaN,4 0변동없음,NaN,곡정보,멋지게 인사하는 법 (Feat. 슬기 of Red Velvet),Zion.T,ZZZ,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생,기타 기능,4
4,NaN,5 0변동없음,NaN,곡정보,고백,양다일,고백,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생,기타 기능,5
5,NaN,6 0변동없음,NaN,곡정보,Good Day (Feat. 팔로알토) (Prod. 코드 쿤스트),pH-1 pH-1,쇼미더머니 777 Episode 1,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생 불가,기타 기능,6
6,NaN,7 0변동없음,NaN,곡정보,시간이 들겠지 (Feat. Colde),로꼬,시간이 들겠지,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생,기타 기능,7
7,NaN,8 1 계단 상승,NaN,곡정보,꿈처럼 내린,다비치,뷰티 인사이드 (JTBC 월화드라마) OST - Part.3,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생,기타 기능,8
8,NaN,9 1 계단 하락,NaN,곡정보,가을 타나 봐,바이브(Vibe),가을 타나 봐,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생 불가,기타 기능,9
9,NaN,10 0변동없음,NaN,곡정보,하루도 그대를 사랑하지 않은 적이 없었다,임창정,하루도 그대를 사랑하지 않은 적이 없었다,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생,기타 기능,10


> ### .head() 옵션

**head 옵션을 사용하면 상위의 지정한 갯수만큼의 필드값을 추출할 수 있다.**

- chart_df.head() # 기본은 5개

- chart_df.head(50) # 50개의 필드 추출

---------------------------

In [17]:
chart_df.head()

,Unnamed: 0,순위,Unnamed: 2,Unnamed: 3,곡,아티스트,앨범,듣기,재생목록,내앨범,다운,영상,기타,좋아요
0,NaN,1 0변동없음,NaN,곡정보,삐삐,아이유(IU),삐삐,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생,기타 기능,1
1,NaN,2 0변동없음,NaN,곡정보,Save (Feat. 팔로알토)(Prod. 코드 쿤스트),루피(Loopy),쇼미더머니 777 Episode 3,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생 불가,기타 기능,2
2,NaN,3 0변동없음,NaN,곡정보,Kiss and Make Up,Dua Lipa(두아 리파) Dua Lipa(두아 리파),Dua Lipa [Complete Edition],듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생 불가,기타 기능,3
3,NaN,4 0변동없음,NaN,곡정보,멋지게 인사하는 법 (Feat. 슬기 of Red Velvet),Zion.T,ZZZ,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생,기타 기능,4
4,NaN,5 0변동없음,NaN,곡정보,고백,양다일,고백,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생,기타 기능,5


> ### .tail() 옵션

**tail 옵션을 사용하면 하위의 지정한 갯수만큼의 필드값을 추출할 수 있다.**

- chart_df.tail() # 기본은 5개

- chart_df.tail(50) # 50개의 필드 추출

---------------------------

In [18]:
chart_df.tail(7)

,Unnamed: 0,순위,Unnamed: 2,Unnamed: 3,곡,아티스트,앨범,듣기,재생목록,내앨범,다운,영상,기타,좋아요
93,NaN,94 23 계단 하락,NaN,곡정보,I'm Fine,방탄소년단,LOVE YOURSELF 結 ‘Answer’,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생 불가,기타 기능,94
94,NaN,95 6 계단 상승,NaN,곡정보,Myself,몬스타엑스(Monsta X),ARE YOU THERE? - The 2nd Album Take.1,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생 불가,기타 기능,95
95,NaN,96 2 계단 상승,NaN,곡정보,시차 (We Are) (Feat. 로꼬 & GRAY),우원재,시차 (We Are),듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생,기타 기능,96
96,NaN,97 10 계단 상승,NaN,곡정보,Roller Coaster,청하,Offset,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생,기타 기능,97
97,NaN,98 4 계단 상승,NaN,곡정보,밤 (Time for the moon night),여자친구(GFRIEND),여자친구 The 6th Mini Album 'Time for the moon night',듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생,기타 기능,98
98,NaN,99 2 계단 하락,NaN,곡정보,Page 0,태연 (TAEYEON) 태연 (TAEYEON),Page 0,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생,기타 기능,99
99,NaN,100 13 계단 상승,NaN,곡정보,행복해야해 (Please...),황치열,Be Myself,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생 불가,기타 기능,100


> ### 레코드값을 이용하여 오름차순, 내림차순하기 `(ascending=True, False)`

-------------------------------

기본옵션은 `True` 로 ascending 옵션을 명시할 필요는 없다.

In [19]:
a = chart_df.sort_values('아티스트', ascending=False) # 아티스트 기준으로 내림차순(황 ~ 특수문자)
a[['곡', '아티스트']]


,곡,아티스트
99,행복해야해 (Please...),황치열
12,어찌 잊으오,황치열
26,바람의 언덕,홍대광
89,LOVE BOMB,프로미스나인
55,비,폴킴(Paul Kim)
16,"모든 날, 모든 순간 (Every day, Every Moment)",폴킴(Paul Kim)
33,헤어지는 중,펀치 (Punch)
98,Page 0,태연 (TAEYEON) 태연 (TAEYEON)
10,넌 (none),크러쉬(Crush)
39,벚꽃연가,첸 (CHEN)


### 좋아요 기준 50개를 넘는 레코드값만 출력하기

----------------------

In [20]:
mask = chart_df['좋아요'] > 50
chart_df[mask]

,Unnamed: 0,순위,Unnamed: 2,Unnamed: 3,곡,아티스트,앨범,듣기,재생목록,내앨범,다운,영상,기타,좋아요
50,NaN,51 1 계단 하락,NaN,곡정보,매트리스,10cm(십센치),4.1,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생,기타 기능,51
51,NaN,52 16 계단 상승,NaN,곡정보,너나 해 (Egotistic),마마무(Mamamoo),RED MOON,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생,기타 기능,52
52,NaN,53 7 계단 하락,NaN,곡정보,밤편지,아이유(IU),밤편지,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생,기타 기능,53
53,NaN,54 7 계단 하락,NaN,곡정보,My Luv,Zion.T,ZZZ,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생 불가,기타 기능,54
54,NaN,55 2 계단 상승,NaN,곡정보,지나오다,닐로(Nil_O),About You,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생 불가,기타 기능,55
55,NaN,56 2 계단 하락,NaN,곡정보,비,폴킴(Paul Kim),비,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생,기타 기능,56
56,NaN,57 16 계단 하락,NaN,곡정보,희미해서 (feat. 헤이즈),이문세,BETWEEN US,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생,기타 기능,57
57,NaN,58 9 계단 상승,NaN,곡정보,그때 헤어지면 돼,로이킴,그때 헤어지면 돼,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생,기타 기능,58
58,NaN,59 6 계단 상승,NaN,곡정보,Heart Attack,몬스타엑스(Monsta X),ARE YOU THERE? - The 2nd Album Take.1,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생 불가,기타 기능,59
59,NaN,60 2 계단 상승,NaN,곡정보,여행,볼빨간사춘기,Red Diary Page.2,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생,기타 기능,60


## 1차적 데이터 구조를 가진 Series

------------------

**Series 를 사용하는 이유 중에 한가지**

<부산지점>

백팩: 10
캐리어: 34
커피: 101

<서울지점>

캐리어: 3
백팩: 55
커피: 23

`* 2지점의 현재 수량은?`

$ total = 부산지점 + 서울지점

$ output: 백팩: 65, 캐리어: 37, 커피: 124

_이와같이 스마트하게 index 를 대조하여 연산이 가능하다._

In [21]:
obj = pd.Series([3, 5, -1, 2]) # index 는 필수로 지정되며, 직접 지정하지 않을 경우에는 아래와 같이 0, 1, ... 와 같이 자동 지정된다.

obj

0    3
1    5
2   -1
3    2
dtype: int64

In [22]:
obj.index

RangeIndex(start=0, stop=4, step=1)

In [23]:
obj = pd.Series([3, 5, -1, 2], index=['순이', '영이', '철수', '군자']) # 필요하다면 직접 인덱스를 지정하자.

obj

순이    3
영이    5
철수   -1
군자    2
dtype: int64

In [24]:
obj.index

Index(['순이', '영이', '철수', '군자'], dtype='object')

In [25]:
obj = pd.Series({'blue': '파란색', 'yellow': '노란색', 'red': '빨간색'}) # dict 타입은 key가 index 가 된다

In [26]:
obj

blue      파란색
yellow    노란색
red       빨간색
dtype: object

### key 를 활용한 value 접근 방법

In [27]:
obj.blue

'파란색'

In [28]:
obj['blue']

'파란색'

In [29]:
obj

blue      파란색
yellow    노란색
red       빨간색
dtype: object

In [30]:
type(chart_df['곡'])

pandas.core.series.Series

In [31]:
chart_df['곡']

0                                        삐삐
1           Save (Feat. 팔로알토)(Prod. 코드 쿤스트)
2                          Kiss and Make Up
3       멋지게 인사하는 법 (Feat. 슬기 of Red Velvet)
4                                        고백
5      Good Day (Feat. 팔로알토) (Prod. 코드 쿤스트)
6                     시간이 들겠지 (Feat. Colde)
7                                    꿈처럼 내린
8                                   가을 타나 봐
9                    하루도 그대를 사랑하지 않은 적이 없었다
10                                 넌 (none)
11                                Shoot Out
12                                   어찌 잊으오
13                               사이렌(Siren)
14                                       물어
15                       이별길 (GOODBYE ROAD)
16    모든 날, 모든 순간 (Every day, Every Moment)
17                            Way Back Home
18      Hate You (Feat. 우원재) (Prod. 코드 쿤스트)
19                                      너에게
20                                Wow Thing
21                                    잘 지내요
22                              

## 크롤링한 데이터 저장하기

------------------------

위에서 크롤링한 데이터를 바로 저장해도 되지만 곡, 아티스트, 앨범만 선택하여 csv, xlsx 파일로 저장해보자

In [32]:
chart_df.columns # 저장할 columns 을 다시 확인해보자

Index(['Unnamed: 0', '순위', 'Unnamed: 2', 'Unnamed: 3', '곡', '아티스트', '앨범', '듣기',
       '재생목록', '내앨범', '다운', '영상', '기타', '좋아요'],
      dtype='object')

In [33]:
result_chart = chart_df[['곡', '아티스트', '앨범']] # 곡, 아티스트, 앨범을 DataFrame 으로 result_chart 에 저장

### xlsx 파일, csv 파일로 저장하기

In [34]:
result_chart.to_excel('bugs_top_100.xlsx') # pypi 에서 제공하는 openpyxl 을 설치하고 엑셀 파일로 저장

In [35]:
result_chart.to_csv('bugs_top_100.csv') # csv 파일로 저장

### 저장한 파일을 다시 불러오기

In [36]:
pd.read_csv('bugs_top_100.csv').head() # 저장한 csv 파일을 불러오기

,Unnamed: 0,곡,아티스트,앨범
0,0,삐삐,아이유(IU),삐삐
1,1,Save (Feat. 팔로알토)(Prod. 코드 쿤스트),루피(Loopy),쇼미더머니 777 Episode 3
2,2,Kiss and Make Up,Dua Lipa(두아 리파) Dua Lipa(두아 리파),Dua Lipa [Complete Edition]
3,3,멋지게 인사하는 법 (Feat. 슬기 of Red Velvet),Zion.T,ZZZ
4,4,고백,양다일,고백


In [37]:
pd.read_excel('bugs_top_100.xlsx').head(10) # xlrd 를 설치 # 엑셀 파일 불러오기

,곡,아티스트,앨범
0,삐삐,아이유(IU),삐삐
1,Save (Feat. 팔로알토)(Prod. 코드 쿤스트),루피(Loopy),쇼미더머니 777 Episode 3
2,Kiss and Make Up,Dua Lipa(두아 리파) Dua Lipa(두아 리파),Dua Lipa [Complete Edition]
3,멋지게 인사하는 법 (Feat. 슬기 of Red Velvet),Zion.T,ZZZ
4,고백,양다일,고백
5,Good Day (Feat. 팔로알토) (Prod. 코드 쿤스트),pH-1 pH-1,쇼미더머니 777 Episode 1
6,시간이 들겠지 (Feat. Colde),로꼬,시간이 들겠지
7,꿈처럼 내린,다비치,뷰티 인사이드 (JTBC 월화드라마) OST - Part.3
8,가을 타나 봐,바이브(Vibe),가을 타나 봐
9,하루도 그대를 사랑하지 않은 적이 없었다,임창정,하루도 그대를 사랑하지 않은 적이 없었다


In [38]:
%ls

Pipfile                   bugs.ipynb                bugs_top_100.xlsx
Pipfile.lock              bugs_top_100.csv          kaggle_titanic_train.csv


## Titanic kaggle 을 활용한 Pandas

--------------------------


Pandas 를 이용하면 현재 생존자가 누구인지, 몇명인지, 성별은 무엇인지 등을 쉽게 파악할 수 있다.

In [39]:
# 데이터분석을 할 경우에는 아래와 같은 양식이 기본적이다.
## shape 로 rows, columns 를 확인 (rows=891 X columns=11)
## head() 로 5줄만 뽑아서 데이터가 정상적으로 추출되었는지 확인

train_df = pd.read_csv('kaggle_titanic_train.csv', index_col=['PassengerId'])
print(train_df.shape)
train_df.head()

(891, 11)


,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


### DataFrame, Series 로 각각 출력해보기

In [40]:
train_df[['Survived']].head() # DataFrame 데이터

,Survived
PassengerId,
1,0
2,1
3,1
4,1
5,0


In [41]:
train_df['Survived'].head() # 참고로 Series 는 하나의 Column 에 대해서만 추출이 가능하다

PassengerId
1    0
2    1
3    1
4    1
5    0
Name: Survived, dtype: int64

### 생존한 사람의 생존여부와 이름을 출력해보기

-------------------

*head() 옵션은 편의상 설정해두었음

In [42]:
train_df[['Name', 'Survived']].head()

,Name,Survived
PassengerId,,
1,"Braund, Mr. Owen Harris",0
2,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1
3,"Heikkinen, Miss. Laina",1
4,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1
5,"Allen, Mr. William Henry",0


In [43]:
train_df['Survived'].head() # Series 데이터

PassengerId
1    0
2    1
3    1
4    1
5    0
Name: Survived, dtype: int64

### index 1번의 정보만 뽑고 싶다면!?

In [44]:
train_df.loc[[1]] # DataFrame

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.25,NaN,S


In [45]:
train_df.loc[1] # Series

Survived                          0
Pclass                            3
Name        Braund, Mr. Owen Harris
Sex                            male
Age                              22
SibSp                             1
Parch                             0
Ticket                    A/5 21171
Fare                           7.25
Cabin                           NaN
Embarked                          S
Name: 1, dtype: object

In [46]:
train_df.loc[[1, 4, 33]] # index 1, 4, 33 의 승객 정보 추출

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.25,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.10,C123,S
33,1,3,"Glynn, Miss. Mary Agatha",female,NaN,0,0,335677,7.75,NaN,Q


In [47]:
train_df.loc[5:13] # index 5~13 까지의 승객 정보 추출

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C
11,1,3,"Sandstrom, Miss. Marguerite Rut",female,4.0,1,1,PP 9549,16.7000,G6,S
12,1,1,"Bonnell, Miss. Elizabeth",female,58.0,0,0,113783,26.5500,C103,S
13,0,3,"Saundercock, Mr. William Henry",male,20.0,0,0,A/5. 2151,8.0500,NaN,S
